In [1]:
import torch 
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

%cd ../

/workspace/Walk_Video_PyTorch/project


In [2]:
from  models.pytorchvideo_models import WalkVideoClassificationLightningModule

/usr/lib/python3.9/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [3]:
from main import get_parameters

opt, unknown = get_parameters()
opt.num_workers = 1

In [4]:
model = WalkVideoClassificationLightningModule(opt)
# model = model.load_from_checkpoint("/workspace/Walk_Video_PyTorch/logs/resnet/0603/checkpoints/epoch=99-step=1800.ckpt")
model.eval()

checkpoint = torch.load("/workspace/Walk_Video_PyTorch/logs/resnet/0708_resnet_depth50/checkpoints/epoch=51-step=3796.ckpt")

model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [5]:
from dataloader.data_loader import WalkDataModule

module = WalkDataModule(opt)
pred_data = module.predict_dataloader()

input_data = next(iter(pred_data))

In [6]:
input_data['video'].shape

input_data["video_name"]

['20190507_2_ASD_lat__V1-0033.mp4',
 '20160927_DHS_lat_V1-0054.mp4',
 '20190909_ASD_lat__V1-0002.mp4',
 '20180723_1_ASD_lat__V1-0014.mp4',
 '20210427_ASD_lat_V1-0011.mp4',
 '20210601_ASD_lat_V1-0002.mp4',
 '20180521_2_ASD_lat__V1-0045.mp4',
 '20190507_1_ASD_lat__V1-0014.mp4',
 '20180903_1_DHS_lat_V1-0037.mp4',
 '20180903_1_DHS_lat_V1-0067.mp4',
 '20180806_2_HipOA_lat_V1-0009.mp4',
 '20210812_DHS_lat_V1-0013.mp4',
 '20180109_LCS_lat_V1-0059.mp4',
 '20210309_ASD_lat__V1-0025.mp4',
 '20190917_ASD_lat__V1-0031.mp4',
 '20190507_1_ASD_lat__V1-0007.mp4']

In [7]:
input_data['label']

tensor([0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0])

In [8]:
preds = model(input_data['video'])

In [9]:
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=1).indices

In [10]:
pred_classes

tensor([[0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0]])

In [11]:
classname = {}

classname[0] = 'asd'
classname[1] = 'asd_not'

In [12]:
real_calss = []

for i in input_data['label'].tolist():
    real_calss.append(classname[i])

In [13]:
real_calss

['asd',
 'asd_not',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd',
 'asd',
 'asd']

In [14]:
# pred_class_names = []
# for num in range(opt._BATCH_SIZE):
#     for i in pred_classes[i]:
#         pred_class_names.append(classname[int(i)])


pred_class_names = [classname[int(i)] for i in pred_classes]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print("real label: %s" % ",".join(real_calss))

Predicted labels: asd, asd_not, asd, asd, asd, asd, asd, asd, asd_not, asd_not, asd_not, asd_not, asd_not, asd, asd, asd
real label: asd,asd_not,asd,asd,asd,asd,asd,asd,asd_not,asd_not,asd_not,asd_not,asd_not,asd,asd,asd


In [ ]:
pred_class_names == real_calss